# 📚 Martín Fierro: Búsqueda de Palabras con Expresiones Regulares

### **🎯 Objetivo del práctico**

La idea principal de este trabajo es amigarnos con las **expresiones regulares (regex)**. Son una herramienta súper potente que nos sirve para buscar patrones en textos. Vamos a usar el `Martín Fierro` de base para aplicar estos conceptos.

-----

## **📖 Consigna 1: Cargar el PDF y separar los párrafos**

### ***💭 Mi explicación***

Lo primero que tenemos que hacer es, lógicamente, traer el texto del libro a nuestro entorno de trabajo. Mediante un enlace URL, lo que voy a hacer es usar una librería para "descargar" el contenido del PDF en memoria y otra para poder leerlo.

Para esto, voy a necesitar estas  librerías:

  * **`requests`**: Para hacer la petición a la URL y traernos el archivo.
  * **`pdfplumber`**: Nos va a permitir abrir el archivo y extraer todo el texto.
  * **`io`**: Esta librería nos va a servir para manejar el contenido del PDF que nos bajamos como si fuera un archivo local, sin tener que guardarlo en la compu.

Una vez que tengamos todo el texto crudo del libro, el siguiente paso es separarlo en párrafos. **CORRECCIÓN IMPORTANTE**: En el Martín Fierro, los párrafos son en realidad **estrofas** (sextinas de 6 versos cada una). Voy a mejorar la detección para que identifique correctamente cada estrofa como un párrafo individual, no páginas completas.

### ***💻 El código***


In [ ]:
# Primero, instalamos las librerías que vamos a necesitar.

!pip install requests
!pip install pdfplumber

import requests
import pdfplumber
import io
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 98.5 MB/s eta 0:00:00


In [ ]:
def cargar_y_separar_parrafos_desde_url(url):
    """
    Esta función se encarga de descargar el PDF desde una URL,
    extraer el texto, limpiarlo y separarlo en párrafos (estrofas).

    CORRECCIÓN: Ahora detecta correctamente las estrofas del Martín Fierro
    como párrafos individuales, no páginas completas.
    """
    print("📥 Descargando el PDF desde la URL...")
    try:
        # Hacemos la petición a la URL para obtener el contenido del PDF
        respuesta = requests.get(url)
        # Verificamos que la descarga haya sido exitosa
        respuesta.raise_for_status()
        print("✅ ¡PDF descargado con éxito!")

        # Usamos io.BytesIO para tratar el contenido binario como un archivo
        archivo_pdf = io.BytesIO(respuesta.content)

        texto_completo = ""
        # Abrimos el PDF con pdfplumber
        with pdfplumber.open(archivo_pdf) as pdf:
            # Juntamos el texto de todas las páginas
            for pagina in pdf.pages:
                texto_pagina = pagina.extract_text()
                if texto_pagina:
                    texto_completo += texto_pagina + "\n"

        print("📝 Texto extraído. Limpiando y separando en estrofas...")

        # 1. Limpieza inicial: removemos los molestos "Página X" del texto
        texto_limpio = re.sub(r'página \d+', '', texto_completo, flags=re.IGNORECASE)

        # 2. Removemos números romanos de los cantos
        texto_limpio = re.sub(r'^[IVXLCDM]+\s*$', '', texto_limpio, flags=re.MULTILINE)

        # 3. Removemos líneas que solo contienen números
        texto_limpio = re.sub(r'^\d+\s*$', '', texto_limpio, flags=re.MULTILINE)

        # 4. CORRECCIÓN PRINCIPAL: Detectar estrofas del Martín Fierro
        # El Martín Fierro está escrito en sextinas (estrofas de 6 versos)
        # Vamos a dividir por líneas y agrupar de a 6 versos
        lineas = texto_limpio.split('\n')

        # Limpiamos líneas vacías y espacios extra
        lineas_limpias = []
        for linea in lineas:
            linea_limpia = linea.strip()
            # Solo agregamos líneas que tengan contenido real (más de 3 caracteres)
            if linea_limpia and len(linea_limpia) > 3:
                # Ignoramos líneas que sean solo títulos o encabezados cortos
                if not re.match(r'^(CANTO|canto|Canto)\s+[IVXLCDM]+$', linea_limpia):
                    lineas_limpias.append(linea_limpia)

        # 5. Agrupamos las líneas en estrofas
        # Detectamos estrofas por patrones de separación
        parrafos_finales = []
        estrofa_actual = []

        for i, linea in enumerate(lineas_limpias):
            # Si la línea parece ser un verso (no muy corta, no es título)
            if len(linea) > 10:
                estrofa_actual.append(linea)

                # Si tenemos 6 versos o encontramos una separación natural
                # (próxima línea muy diferente o fin de archivo)
                if len(estrofa_actual) == 6:
                    parrafos_finales.append('\n'.join(estrofa_actual))
                    estrofa_actual = []
                elif i < len(lineas_limpias) - 1:
                    # Verificamos si hay un salto natural (línea siguiente muy corta o diferente)
                    siguiente = lineas_limpias[i + 1] if i + 1 < len(lineas_limpias) else ""
                    if len(siguiente) < 10 or re.match(r'^[A-Z]{3,}', siguiente):
                        if len(estrofa_actual) >= 4:  # Mínimo 4 versos para considerarlo estrofa
                            parrafos_finales.append('\n'.join(estrofa_actual))
                            estrofa_actual = []

        # Si quedó alguna estrofa sin procesar
        if len(estrofa_actual) >= 4:
            parrafos_finales.append('\n'.join(estrofa_actual))

        # 6. Método alternativo: si no detectamos suficientes estrofas,
        # usamos separación por doble salto de línea
        if len(parrafos_finales) < 50:  # El Martín Fierro tiene cientos de estrofas
            print("⚠️ Usando método alternativo de detección de párrafos...")
            # Volvemos al texto limpio y lo separamos por doble salto
            parrafos_alt = re.split(r'\n\s*\n', texto_limpio)
            parrafos_finales = []
            for p in parrafos_alt:
                p_limpio = p.strip()
                # Solo agregamos si tiene contenido sustancial (más de 20 caracteres)
                if p_limpio and len(p_limpio) > 20:
                    parrafos_finales.append(p_limpio)

        print(f"✨ ¡Proceso completado! Se encontraron {len(parrafos_finales)} párrafos/estrofas.")
        return parrafos_finales

    except requests.exceptions.RequestException as e:
        print(f"❌ Ocurrió un error al descargar el archivo: {e}")
        return []
    except Exception as e:
        print(f"❌ Ocurrió un error al procesar el PDF: {e}")
        return []

# La URL del Martín Fierro
url_martin_fierro = "https://www.argentina.gob.ar/sites/default/files/hernandez_jose_-_el_gaucho_martin_fierro.pdf"

# Llamamos a la función para obtener la lista de párrafos
lista_parrafos_originales = cargar_y_separar_parrafos_desde_url(url_martin_fierro)

# Para chusmear, imprimimos algunos párrafos para ver cómo quedaron
if lista_parrafos_originales:
    print("\n--- 📋 Ejemplo de los primeros párrafos/estrofas ---")
    for i in range(min(3, len(lista_parrafos_originales))):
        print(f"\n🎵 Párrafo/Estrofa {i+1}:")
        print(lista_parrafos_originales[i])
        print("-" * 50)


📥 Descargando el PDF desde la URL...
✅ ¡PDF descargado con éxito!
📝 Texto extraído. Limpiando y separando en estrofas...
✨ ¡Proceso completado! Se encontraron 388 párrafos/estrofas.

--- 📋 Ejemplo de los primeros párrafos/estrofas ---

🎵 Párrafo/Estrofa 1:
Martín Fierro
José Hernández
Recursos de dominio público
José Hernández
El Gaucho Martín Fierro
José Hernández (1834 - 1886)
--------------------------------------------------

🎵 Párrafo/Estrofa 2:
Imágenes de dominio público. Fuente:
http://upload.wikimedia.org/wikipedia/commons/8/8c/Jos%C3%A9_Hern%C3%A1ndez_Argentino.jpg
http://upload.wikimedia.org/wikipedia/commons/3/30/El_Gaucho_Mart%C3%ADn_Fierro_2.jpg
http://upload.wikimedia.org/wikipedia/commons/4/44/El_Gaucho_Mart%C3%ADn_Fierro_3.jpg
http://upload.wikimedia.org/wikipedia/commons/4/47/El_Gaucho_Mart%C3%ADn_Fierro_1.jpg
El Gaucho Martín Fierro
--------------------------------------------------

🎵 Párrafo/Estrofa 3:
Aquí me pongo a cantar
al compás de la vigüela,
que el hombre q



## **🔤 Consigna 2: Preprocesar el texto**

### ***💭 Mi explicación***

Ahora, la consigna pedía que pase todo a minúsculas. Esto es un paso clave en el preprocesamiento de texto. ¿Por qué? Porque si buscamos la palabra "gaucho", queremos que la búsqueda encuentre "Gaucho", "GAUCHO" o "gaucho". Al estandarizar todo a minúsculas, nos aseguramos de que la búsqueda no sea sensible a mayúsculas y minúsculas y no se nos escape ningún resultado.

Lo que voy a hacer es: recorrer la lista de párrafos que creé antes y a cada uno le voy a aplicar un método que lo convierte a minúsculas. Voy a guardar el resultado en una nueva lista para mantener los originales por las dudas.

### ***💻 El código***


In [ ]:
def preprocesar_parrafos(parrafos):
    """
    Recibe una lista de párrafos y los convierte todos a minúsculas.
    """
    print("\n🔧 Preprocesando el texto: convirtiendo todo a minúsculas...")
    # Usamos una "list comprehension" para hacerlo más cortito y prolijo.
    # Recorre cada párrafo 'p' en la lista 'parrafos' y le aplica el método .lower()
    parrafos_en_minusculas = [p.lower() for p in parrafos]
    print("✅ ¡Procesamiento terminado!")
    return parrafos_en_minusculas

# Aplicamos la función a nuestra lista de párrafos
lista_parrafos_minusculas = preprocesar_parrafos(lista_parrafos_originales)

# Verificamos el resultado con algunos párrafos
if lista_parrafos_minusculas:
    print("\n--- 📋 Ejemplo de párrafos en minúsculas ---")
    for i in range(min(2, len(lista_parrafos_minusculas))):
        print(f"\n🔡 Párrafo {i+1} (minúsculas):")
        print(lista_parrafos_minusculas[i])
        print("-" * 50)


🔧 Preprocesando el texto: convirtiendo todo a minúsculas...
✅ ¡Procesamiento terminado!

--- 📋 Ejemplo de párrafos en minúsculas ---

🔡 Párrafo 1 (minúsculas):
martín fierro
josé hernández
recursos de dominio público
josé hernández
el gaucho martín fierro
josé hernández (1834 - 1886)
--------------------------------------------------

🔡 Párrafo 2 (minúsculas):
imágenes de dominio público. fuente:
http://upload.wikimedia.org/wikipedia/commons/8/8c/jos%c3%a9_hern%c3%a1ndez_argentino.jpg
http://upload.wikimedia.org/wikipedia/commons/3/30/el_gaucho_mart%c3%adn_fierro_2.jpg
http://upload.wikimedia.org/wikipedia/commons/4/44/el_gaucho_mart%c3%adn_fierro_3.jpg
http://upload.wikimedia.org/wikipedia/commons/4/47/el_gaucho_mart%c3%adn_fierro_1.jpg
el gaucho martín fierro
--------------------------------------------------


## **🔍 Consigna 3 y 4: Búsqueda de palabras y creación de una función**

### ***💭 Mi explicación***

Ahora vamos a usar **expresiones regulares** para buscar palabras dentro de los párrafos que ya procesamos. La librería que se usa en Python para esto es `re`.

La idea es crear una función que:
1. Reciba la palabra a buscar y el listado de párrafos
2. Convierta la palabra a minúsculas
3. Use expresiones regulares para buscar en cada párrafo
4. **IMPORTANTE**: Devuelva una **lista/vector** con los párrafos donde aparece la palabra

### ***💻 El código***

In [ ]:
def buscar_palabra_con_regex(palabra_a_buscar, listado_de_parrafos):
    """
    Busca una palabra en una lista de párrafos usando expresiones regulares.

    Args:
        palabra_a_buscar (str): La palabra o término que se quiere encontrar.
        listado_de_parrafos (list): La lista de párrafos donde buscar.

    Returns:
        list: Una LISTA/VECTOR con los párrafos donde aparece dicha palabra.
              (Cumpliendo con el requisito de la consigna 4)
    """
    # Nos aseguramos de que la palabra a buscar esté en minúsculas
    palabra_minusculas = palabra_a_buscar.lower()
    print(f"\n🔎 Buscando la palabra '{palabra_minusculas}' en el texto...")

    # Esta es la LISTA/VECTOR que va a contener los párrafos encontrados
    parrafos_encontrados = []
    contador_apariciones = 0

    # Recorremos cada párrafo de la lista que nos pasaron
    for indice, parrafo in enumerate(listado_de_parrafos):
        # Usamos re.search para ver si la palabra está en el párrafo.
        # \b es un "ancla" de regex que significa "límite de palabra".
        # Lo usamos para buscar la palabra exacta y no sub-palabras.
        if re.search(r'\b' + re.escape(palabra_minusculas) + r'\b', parrafo):
            parrafos_encontrados.append(parrafo)
            contador_apariciones += 1

    print(f"✅ Se encontraron {len(parrafos_encontrados)} párrafos que contienen la palabra '{palabra_minusculas}'.")

    # Retornamos la LISTA/VECTOR con los párrafos donde aparece la palabra
    return parrafos_encontrados

# --- 🧪 ¡Probemos la función! ---

# Primero, busquemos una palabra común en el Martín Fierro, como "gaucho"
palabra1 = "gaucho"
vector_parrafos_gaucho = buscar_palabra_con_regex(palabra1, lista_parrafos_minusculas)

# Imprimimos algunos de los párrafos encontrados para verificar
if vector_parrafos_gaucho:
    print(f"\n--- 📚 Mostrando hasta 3 párrafos/estrofas con la palabra '{palabra1}' ---")
    for i, p in enumerate(vector_parrafos_gaucho[:3]):
        print(f"\n📖 Resultado {i+1}:")
        # Para que se vea mejor, resaltamos la palabra encontrada
        p_resaltado = re.sub(
            r'(\b' + re.escape(palabra1) + r'\b)',
            r'**\1**',
            p,
            flags=re.IGNORECASE
        )
        print(p_resaltado)
        print("-" * 50)

# Ahora, probemos con otra palabra, por ejemplo, "pampa"
palabra2 = "pampa"
vector_parrafos_pampa = buscar_palabra_con_regex(palabra2, lista_parrafos_minusculas)

if vector_parrafos_pampa:
    print(f"\n--- 📚 Mostrando hasta 3 párrafos/estrofas con la palabra '{palabra2}' ---")
    for i, p in enumerate(vector_parrafos_pampa[:3]):
        print(f"\n📖 Resultado {i+1}:")
        p_resaltado = re.sub(
            r'(\b' + re.escape(palabra2) + r'\b)',
            r'**\1**',
            p,
            flags=re.IGNORECASE
        )
        print(p_resaltado)
        print("-" * 50)

# Probemos con "justicia" también
palabra3 = "justicia"
vector_parrafos_justicia = buscar_palabra_con_regex(palabra3, lista_parrafos_minusculas)

if vector_parrafos_justicia:
    print(f"\n--- 📚 Mostrando hasta 2 párrafos/estrofas con la palabra '{palabra3}' ---")
    for i, p in enumerate(vector_parrafos_justicia[:2]):
        print(f"\n📖 Resultado {i+1}:")
        p_resaltado = re.sub(
            r'(\b' + re.escape(palabra3) + r'\b)',
            r'**\1**',
            p,
            flags=re.IGNORECASE
        )
        print(p_resaltado)
        print("-" * 50)

# --- 📊 Resumen final ---
print("\n" + "="*60)
print("📊 RESUMEN DE LA BÚSQUEDA")
print("="*60)
print(f"Total de párrafos/estrofas procesados: {len(lista_parrafos_minusculas)}")
print(f"Párrafos con '{palabra1}': {len(vector_parrafos_gaucho)}")
print(f"Párrafos con '{palabra2}': {len(vector_parrafos_pampa)}")
print(f"Párrafos con '{palabra3}': {len(vector_parrafos_justicia)}")
print("="*60)


🔎 Buscando la palabra 'gaucho' en el texto...
✅ Se encontraron 55 párrafos que contienen la palabra 'gaucho'.

--- 📚 Mostrando hasta 3 párrafos/estrofas con la palabra 'gaucho' ---

📖 Resultado 1:
martín fierro
josé hernández
recursos de dominio público
josé hernández
el **gaucho** martín fierro
josé hernández (1834 - 1886)
--------------------------------------------------

📖 Resultado 2:
imágenes de dominio público. fuente:
http://upload.wikimedia.org/wikipedia/commons/8/8c/jos%c3%a9_hern%c3%a1ndez_argentino.jpg
http://upload.wikimedia.org/wikipedia/commons/3/30/el_gaucho_mart%c3%adn_fierro_2.jpg
http://upload.wikimedia.org/wikipedia/commons/4/44/el_gaucho_mart%c3%adn_fierro_3.jpg
http://upload.wikimedia.org/wikipedia/commons/4/47/el_gaucho_mart%c3%adn_fierro_1.jpg
el **gaucho** martín fierro
--------------------------------------------------

📖 Resultado 3:
soy **gaucho**, y entiéndalo
como mi lengua lo esplica:
para mí la tierra es chica
y pudiera ser mayor;
ni la víbora me pica
n